# Getting Started - Multivariate Linear Regression with R
Revision 1.0

## Multivariate Linear Regression warmup with lm() with mtcars dataset

**Web Resources**
- https://www.rdocumentation.org/packages/datasets/versions/3.6.2/topics/mtcars
- https://stats.stackexchange.com/questions/44838/how-are-the-standard-errors-of-coefficients-calculated-in-a-regression
- https://gattonweb.uky.edu/sheather/book/

In [1]:
head(mtcars)

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Mazda RX4,21.0,6,160,110,3.90,2.620,16.46,0,1,4,4
Mazda RX4 Wag,21.0,6,160,110,3.90,2.875,17.02,0,1,4,4
Datsun 710,22.8,4,108,93,3.85,2.320,18.61,1,1,4,1
Hornet 4 Drive,21.4,6,258,110,3.08,3.215,19.44,1,0,3,1
Hornet Sportabout,18.7,8,360,175,3.15,3.440,17.02,0,0,3,2
Valiant,18.1,6,225,105,2.76,3.460,20.22,1,0,3,1


In [2]:
linreg.fit <- lm(mtcars$hp~mtcars$mpg + mtcars$wt + mtcars$qsec)
summary(linreg.fit)


Call:
lm(formula = mtcars$hp ~ mtcars$mpg + mtcars$wt + mtcars$qsec)

Residuals:
    Min      1Q  Median      3Q     Max 
-48.392 -14.285  -6.816  12.320  97.624 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  494.570     78.276   6.318 7.81e-07 ***
mtcars$mpg    -2.700      2.269  -1.190   0.2442    
mtcars$wt     25.042     12.892   1.942   0.0622 .  
mtcars$qsec  -20.966      3.865  -5.425 8.68e-06 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 31.73 on 28 degrees of freedom
Multiple R-squared:  0.8066,	Adjusted R-squared:  0.7859 
F-statistic: 38.93 on 3 and 28 DF,  p-value: 4.017e-10


In [37]:
t(sqrt(diag(vcov(linreg.fit))))

(Intercept),mtcars$mpg,mtcars$wt,mtcars$qsec
78.27597,2.269247,12.89193,3.864521


## Multivariate Linear Regression warmup through linear algebra with mtcars dataset

In [16]:
# Pseudo Inverse compute function
pseudo.inv <- function(M){
  solve(t(M)%*%M)%*%t(M)
}

# Fit Model for simple linear regresion
linear.model <- function(t) {
  c(t, 1)
}

# Sum of square through dot product
sum.squares <- function(v){
    t(matrix(v)) %*% matrix(v)
}

In [26]:
A <- cbind(mtcars$mpg, mtcars$wt, mtcars$qsec, 1) # Design or measurements matrix
linear.coefs <- pseudo.inv(A)%*%matrix(mtcars$hp) # Coefficients Estimation
rownames(linear.coefs) <- c("mpg", "wt", "qsec", "intercept")
linear.coefs

mpg,-2.699598
wt,25.042342
qsec,-20.965780
intercept,494.569588


In [121]:
sigma.sq <- sum((matrix(mtcars$hp) - A%*%linear.coefs)^2)/(nrow(A) - ncol(A)) # Sigma squared
m.varcovar <- sigma.sq*chol2inv(chol(t(A)%*%A))                               # variance-covariance matrix using QR decomposition
m.stderrors <- sqrt(diag(m.varcovar))                                         # Coefficients Std Error Vector
linear.results <- data.frame(coeffs = linear.coefs, stderr = m.stderrors, tvals = linear.coefs/m.stderrors)
linear.results$pvalues <- 2*pt(abs(linear.results$tvals), nrow(A) - ncol(A), lower.tail = FALSE)
linear.results

,coeffs,stderr,tvals,pvalues
,<dbl>,<dbl>,<dbl>,<dbl>
mpg,-2.699598,2.269247,-1.189645,2.441762e-01
wt,25.042342,12.891932,1.942482,6.219801e-02
qsec,-20.965780,3.864521,-5.425195,8.684578e-06
intercept,494.569588,78.275972,6.318281,7.805802e-07


In [123]:
linear.results$lwr.coeff <- linear.results$coeffs - abs(qt(0.025, nrow(A)-ncol(A)))*linear.results$stderr
linear.results$upr.coeff <- linear.results$coeffs + abs(qt(0.025, nrow(A)-ncol(A)))*linear.results$stderr
linear.results

,coeffs,stderr,tvals,pvalues,lwr.coeff,upr.coeff
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
mpg,-2.699598,2.269247,-1.189645,2.441762e-01,-7.347938,1.948743
wt,25.042342,12.891932,1.942482,6.219801e-02,-1.365583,51.450266
qsec,-20.965780,3.864521,-5.425195,8.684578e-06,-28.881891,-13.049668
intercept,494.569588,78.275972,6.318281,7.805802e-07,334.228528,654.910648


In [127]:
y.hat <- linear.coefs[1]*mtcars$mpg + linear.coefs[2]*mtcars$wt + linear.coefs[3]*mtcars$qsec + linear.coefs[4]
summary(mtcars$hp - y.hat)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
-48.392 -14.285  -6.816   0.000  12.320  97.624 

In [128]:
SS.total  <- sum.squares(mtcars$hp - mean(mtcars$hp))      # Regression total sum of squares
r.squared <- sum.squares(y.hat - mean(mtcars$hp))/SS.total # Is the same as SS.reg/SS.total = explained var/total var
r.squared

0.8066039


In [135]:
SS.err   <- sum.squares(mtcars$hp - y.hat) # Residuals sum of squares
SS.reg   <- SS.total - SS.err              # Regression sum of squares
MS.reg <- SS.reg/(ncol(A) - 1)             # Mean squares regression for multiple predictors
MS.res <- sum.squares(mtcars$hp - y.hat)/(nrow(mtcars) - ncol(A))  # Mean squares residuals
f.stat <- MS.reg/MS.res              # The ratio of explained to unexplained variability
f.stat

38.92686


In [136]:
pf(f.stat, ncol(A) - 1, nrow(A) - ncol(A), lower.tail = FALSE) # df1 = multiple predictors

4.017096e-10
